# Linear Regression

* Linearity
* Independence
* Normality

Para un conjunto de puntos ${x_i}$ tenemos el modelo lineal 
\begin{equation}
 \hat{y}_i = a_0 + a_1 x_i
\end{equation}

definimos el error asociado al ajuste del modelo lineal como $Res = |y_i - \hat{y_i}|$. Donde el modelo de regresion lineal se obtiene a partir de minimizar la suma de los residuos al cuadrado $\sum Res² = \sum |y_i - \hat{y_i}|^2$


#### Evaluating Linear regression model 
* Mean Absolut Error (MAE)
\begin{equation}
MAE = \sum \frac{|y_i - \hat{y_i}|}{N}
\end{equation}

* Mean Squared Error (MSE)
\begin{equation}
MSE = \sum \frac{|y_i - \hat{y_i}|^2}{N}
\end{equation}


* Root Mean Squared Error (RMSE)
\begin{equation}
RMSE = \sqrt{\sum \frac{|y_i - \hat{y_i}|^2}{N}}
\end{equation}

* Coefficient of determination $R^2$

\begin{equation}
R^2 = 1 - \frac{RSS}{TSS}
\end{equation}

donde $RSS = \sum |y_i - \hat{y_i}|^2$ y $TSS = \sum (y_i - \bar{y})^2
$
